In [1]:
# loading required packages

from scrapy import Selector
import requests
import pandas as pd

# 1. Web scraping from webpage: www.euro.com.pl

In [2]:
# importing names, prices and parameters of microwaves from webpage euro.com.pl

url_euroagd = 'https://www.euro.com.pl/kuchenki-mikrofalowe,'

nazwy_euroagd=[]
ceny_euroagd=[]
parametry_euroagd_all=[]
wartosci_euroagd_all=[]


def parse_page(number):
    page = requests.get(f'{url_euroagd}strona-{number}.bhtml').content
    sel = Selector (text = page)
    for produkt in sel.xpath('//div[@class="product-for-list"]//div[@class="product-header"]//a[@class="js-save-keyword"]/text()'):
        nazwa_euroagd = produkt.get().strip()
        nazwy_euroagd.append(nazwa_euroagd)
        
    for cena in sel.xpath('//div[@class="product-for-list"]//div[@class="price-normal selenium-price-normal"]/text()'):
        cena_euroagd = cena.get().strip().replace(u'\xa0', u'').rstrip("zł")
        ceny_euroagd.append(cena_euroagd)
        
    for parametry in sel.xpath('//div[@class="product-info"]//div[@class="product-attributes"]//div[@class="attributes-row"]//a[@class="js-dictionary"]/text()'):
        parametr_euroagd = parametry.get().strip()
        parametry_euroagd_all.append(parametr_euroagd)
        
    for wartosc in sel.xpath('//div[@class="product-info"]//div[@class="product-attributes"]//div[@class="attributes-row"]//span[@class="attribute-value"]/text()'):
        wartosc_euroagd = wartosc.get().strip()
        wartosci_euroagd_all.append(wartosc_euroagd)
        
# searching for all subpages of microwaves category
        
for number in range(1, 4):
    parse_page(number)

In [3]:
# changing type of elements in ceny_euroagd (prices) from string to float

for i in range(0,len(ceny_euroagd)):
        ceny_euroagd[i] = float(ceny_euroagd[i])
        
ceny_euroagd = ceny_euroagd[0::2]

In [4]:
# creating data frame with name of products, prices, and assigning product id to each product (which will allow to join them with their parameters on later stage)

euroagd = pd.DataFrame(
    {'Nazwa': nazwy_euroagd,
     'Cena (w zł)': ceny_euroagd,
    })
euroagd['id'] = euroagd.index+1
euroagd.head()

,Nazwa,Cena (w zł),id
0,Samsung GE83X,429.0,1
1,Whirlpool MWP203M,389.0,2
2,Sharp R200(IN)W,269.0,3
3,Amica AMGF20M1GS,275.0,4
4,Sharp R760S,449.0,5


In [5]:
# creating data frame with parametres and their values

euroagd_parametr = pd.DataFrame(
            {'Parametr': parametry_euroagd_all,
            'Wartosc': wartosci_euroagd_all})
euroagd_parametr.head()

,Parametr,Wartosc
0,Wymiary (GxSxW),"35,4 x 48,9 x 27,5 cm"
1,Pojemność,23 litry
2,Funkcje podstawowe,"gotowanie, grill, podgrzewanie, rozmrażanie"
3,Moc mikrofal,800 W
4,Moc grilla,1100 W


In [6]:
# assigning product id to each parameter

id_obiektu = 0
ids = []
for i in range(len(euroagd_parametr)):
    row = euroagd_parametr.iloc[[i]]
    name_column = list(row['Parametr'])[0]
    if  name_column == "Wymiary (GxSxW)":
        id_obiektu += 1
    
    ids.append(id_obiektu)
    
euroagd_parametr_id = pd.DataFrame(
            {'Parametr': parametry_euroagd_all,
            'Wartosc': wartosci_euroagd_all,
            'id': ids})
euroagd_parametr_id.head()

,Parametr,Wartosc,id
0,Wymiary (GxSxW),"35,4 x 48,9 x 27,5 cm",1
1,Pojemność,23 litry,1
2,Funkcje podstawowe,"gotowanie, grill, podgrzewanie, rozmrażanie",1
3,Moc mikrofal,800 W,1
4,Moc grilla,1100 W,1


In [7]:
# joining parameters table with product table

euroagd_merged = (pd.merge(euroagd, euroagd_parametr_id, on= "id", how='left'))
euroagd_merged.head()

,Nazwa,Cena (w zł),id,Parametr,Wartosc
0,Samsung GE83X,429.0,1,Wymiary (GxSxW),"35,4 x 48,9 x 27,5 cm"
1,Samsung GE83X,429.0,1,Pojemność,23 litry
2,Samsung GE83X,429.0,1,Funkcje podstawowe,"gotowanie, grill, podgrzewanie, rozmrażanie"
3,Samsung GE83X,429.0,1,Moc mikrofal,800 W
4,Samsung GE83X,429.0,1,Moc grilla,1100 W


In [8]:
# final table containing all microwaves available on euro.com.pl webpage with their prices and parameters

pd.set_option("max_rows", 150)
euroagd_final = euroagd_merged.pivot_table(index = ['Nazwa', 'Cena (w zł)'],
                                                    columns = 'Parametr', 
                                                    values = 'Wartosc',
                                                    fill_value = 'brak informacji',
                                                    aggfunc = 'first')#.reset_index()
euroagd_final

,Parametr,Funkcje podstawowe,Moc grilla,Moc mikrofal,Pojemność,Wymiary (GxSxW)
Nazwa,Cena (w zł),,,,,
Amica AMG20M70GBIV,289.0,"grill, podgrzewanie",900 W,700 W,20 litrów,"33,5 x 45,2 x 26,2 cm"
Amica AMG20M70V,249.0,podgrzewanie,brak informacji,700 W,20 litrów,"32 x 45,2 x 26,2 cm"
Amica AMGF17M2B,229.0,podgrzewanie,brak informacji,600 W,17 litrów,"31,5 x 45,2 x 26,2 cm"
Amica AMGF20E1GB,329.0,"grill, podgrzewanie",900 W,700 W,20 litrów,"33,3 x 45,2 x 26,2 cm"
Amica AMGF20E1GI,349.0,"gotowanie, grill, podgrzewanie, rozmrażanie",900 W,700 W,20 litrów,"31,2 x 45,2 x 26,2 cm"
Amica AMGF20M1GS,275.0,"grill, podgrzewanie",900 W,700 W,20 litrów,"35,2 x 45,2 x 26,2 cm"
Amica AMGF20M1GW,269.0,"gotowanie, grill, podgrzewanie",1000 W,800 W,20 litrów,"38 x 45,2 x 26,2 cm"
Amica AMGF23E1GB,309.0,"gotowanie, grill, podgrzewanie",1000 W,900 W,23 litry,"37,2 x 48,3 x 28,1 cm"
Amica AMGF23E1GS,329.0,"grill, podgrzewanie",1000 W,900 W,23 litry,"39,6 x 48,3 x 28,1 cm"


# 2. Web scraping from webpage: www.morele.net

In [22]:
# importing names, prices and parameters of microwaves from webpage morele.net

url_morele = 'https://www.morele.net/kategoria/kuchenki-mikrofalowe-86/,,,,,,,,0,,,,/'

nazwy_morele=[]
ceny_morele=[]
parametry_morele_all=[]
wartosci_morele_all=[]


def parse_page_morele(number):
    page = requests.get(f'{url_morele}{number}/').content
    sel = Selector (text = page)
    
    for produkt in sel.xpath('//div[@class="cat-list-products"]//p[@class="cat-product-name"]/a[@class="productLink"]/text()'):
        nazwa_morele = produkt.get().strip().lstrip("Kuchenka mikrofalowa ").replace(" Kuchenka mikrofalowa", "")
        nazwy_morele.append(nazwa_morele)
        
    for cena in sel.xpath('//div[@class="cat-list-products"]//div[@class="cat-product-inside"]//*[contains(@class, "price-box")]/div[@class="price-new"]/text()'):
        cena_morele = cena.get().strip().lstrip("od ").rstrip(" zł").replace(",", ".").replace(" ","")
        ceny_morele.append(cena_morele)
        
    for parametry in sel.xpath('//div[@class="cat-list-products"]//div[@class="cat-product-center-inside"]//text()'):
        parametry_morele = parametry.get().strip()
        parametry_morele_all.append(parametry_morele)
        
    for wartosci in sel.xpath('//div[@class="cat-list-products"]//div[@class="cat-product-features"]/div[@class="cat-product-feature"]/b/text()'):
        wartosci_morele = wartosci.get().strip()
        wartosci_morele_all.append(wartosci_morele)

for number in range(1, 7):
    parse_page_morele(number)

In [23]:
# changing type of elements in ceny_morele (prices) from string to float


ceny_morele_new=[]

for cena in ceny_morele:
    if len(cena)>0:
        ceny_morele_new.append(cena)
        
for i in range(0,len(ceny_morele_new)):
        ceny_morele_new[i] = float(ceny_morele_new[i])

In [24]:
# choosing appriopriate values from list of parameters

parametry_morele_all_new=[]

for parametr in parametry_morele_all:
    if len(parametr)>0:
        parametry_morele_all_new.append(parametr)
        
parametry_morele_all_new2 = []

for parametr in parametry_morele_all_new:
    if parametr.startswith("Kuchenka mikrofalowa"):
        parametr1 = "Nowy obiekt"
        parametry_morele_all_new2.append(parametr1)
    elif parametr == "Funkcje podstawowe:":
        parametry_morele_all_new2.append(parametr)
    elif parametr == "Moc mikrofali [W]:":
        parametry_morele_all_new2.append(parametr)
    elif parametr == "Pojemność [l]:":
        parametry_morele_all_new2.append(parametr)
    elif parametr == "Sterowanie:":
        parametry_morele_all_new2.append(parametr)

In [25]:
# creating data frame with name of products, prices, and assigning product id to each product (which will allow to join them with their parameters on later stage)

morele = pd.DataFrame(
    {'Nazwa': nazwy_morele,
     'Cena (w zł)': ceny_morele_new,
    })
morele['id'] = morele.index+1
morele.head()

,Nazwa,Cena (w zł),id
0,Beko MOC20100W,242.99,1
1,Amica AMGF17M1GW,267.00,2
2,Samsung MS23K3513AS,408.99,3
3,Samsung ME83X,365.78,4
4,Amica AMG20E70GSV,274.75,5


In [26]:
# assigning product id to each parameter

morele_parametr = pd.DataFrame(
            {'Parametr': parametry_morele_all_new2})

id_obiektu_morele = 0
ids_morele = []
for i in range(len(parametry_morele_all_new2)):
    row = morele_parametr.iloc[[i]]
    name_column = list(row['Parametr'])[0]
    if  name_column == "Nowy obiekt":
        id_obiektu_morele += 1
    
    ids_morele.append(id_obiektu_morele)
    
morele_parametr_id = pd.DataFrame(
    {'Parametr': parametry_morele_all_new2, 'id': ids_morele})
morele_parametr_id = morele_parametr_id[morele_parametr_id.Parametr != "Nowy obiekt"]

morele_parametr_id['Wartosc'] = wartosci_morele_all
morele_parametr_id.head()

,Parametr,id,Wartosc
1,Funkcje podstawowe:,1,"Rozmrażanie,Standard"
2,Moc mikrofali [W]:,1,700
3,Pojemność [l]:,1,20
4,Sterowanie:,1,Mechaniczne
6,Funkcje podstawowe:,2,Grill


In [27]:
# joining parameters table with product table

morele_merged = (pd.merge(morele, morele_parametr_id, on= "id", how='left'))
morele_merged.head()

,Nazwa,Cena (w zł),id,Parametr,Wartosc
0,Beko MOC20100W,242.99,1,Funkcje podstawowe:,"Rozmrażanie,Standard"
1,Beko MOC20100W,242.99,1,Moc mikrofali [W]:,700
2,Beko MOC20100W,242.99,1,Pojemność [l]:,20
3,Beko MOC20100W,242.99,1,Sterowanie:,Mechaniczne
4,Amica AMGF17M1GW,267.00,2,Funkcje podstawowe:,Grill


In [28]:
# final table containing all microwaves available on euro.com.pl webpage with their prices and parameters

pd.set_option("max_rows", 150)
morele_final = morele_merged.pivot_table(index = ['Nazwa', 'Cena (w zł)'],
                                                    columns = 'Parametr', 
                                                    values = 'Wartosc',
                                                    fill_value = 'brak informacji',
                                                    aggfunc = 'first')

morele_final

,Parametr,Funkcje podstawowe:,Moc mikrofali [W]:,Pojemność [l]:,Sterowanie:
Nazwa,Cena (w zł),,,,
Amica AMG17M70V,245.00,Rozmrażanie,700,17,Mechaniczne
Amica AMG20E70GSV,274.75,"Rozmrażanie,Grill",700,20,Elektroniczne
Amica AMG20M70GBIV,278.99,"Rozmrażanie,Grill",700,20,Mechaniczne
Amica AMG20M70V,241.99,"Rozmrażanie,Standard",700,20,Mechaniczne
Amica AMGF17M1GW,267.00,Grill,700,17,Mechaniczne
Amica AMGF17M1W,247.00,"Rozmrażanie,Standard",700,17,Mechaniczne
Amica AMGF20E1GB,299.00,"Rozmrażanie,Standard,Grill",700,20,Elektroniczne
Amica AMGF20M1GS,294.80,"Rozmrażanie,Grill",700,20,Mechaniczne
Amica AMGF23E1GB,359.00,Grill,900,23,Elektroniczne


# 3. Comparing prices of microwaves available on both webpages

In [29]:
# extracting only names and prices from data frames for euro.com.pl and morele.net and renaming price columns

euroagd_porownanie = euroagd_final.reset_index()[['Nazwa', 'Cena (w zł)']]
morele_porownanie = morele_final.reset_index()[['Nazwa', 'Cena (w zł)']]

euroagd_porownanie = euroagd_porownanie.rename(columns={'Cena (w zł)':'Cena (w zł) euroagd'})
morele_porownanie = morele_porownanie.rename(columns={'Cena (w zł)':'Cena (w zł) morele'})

In [30]:
# final data frame containing all microwaves available on both webpages, their prices, as well as relative and absolute differences in price between webpages

Porownanie = pd.merge(euroagd_porownanie, morele_porownanie, on="Nazwa", how="inner")

Porownanie['Różnica'] = Porownanie.apply(lambda x: x['Cena (w zł) euroagd'] - x['Cena (w zł) morele'], axis=1)
Porownanie['Różnica (w %)'] = Porownanie.apply(lambda x: round((x['Cena (w zł) euroagd'] - x['Cena (w zł) morele']) / x['Cena (w zł) euroagd'] *100,2), axis=1)
Porownanie

Parametr,Nazwa,Cena (w zł) euroagd,Cena (w zł) morele,Różnica,Różnica (w %)
0,Amica AMG20M70GBIV,289.0,278.99,10.01,3.46
1,Amica AMG20M70V,249.0,241.99,7.01,2.82
2,Amica AMGF20E1GB,329.0,299.00,30.00,9.12
3,Amica AMGF20M1GS,275.0,294.80,-19.80,-7.20
4,Amica AMGF23E1GB,309.0,359.00,-50.00,-16.18
5,Amica AMGF23E1GS,329.0,368.99,-39.99,-12.16
6,Amica AMMF20M1GI,349.0,298.20,50.80,14.56
7,LG MS23NECBW,499.0,529.22,-30.22,-6.06
8,Samsung GE83M,389.0,389.00,0.00,0.00
9,Samsung ME83M,359.0,329.87,29.13,8.11
